# How to create simple dashboard with widgets in Python Bokeh

* Ref: [https://coderzcolumn.com/tutorials/data-science/simple-dashboard-with-widgets-python-bokeh?fbclid=IwAR1qkq4v3yphKh45LFoZJQQOJGxkgXOGxumn5QKpBeDwd9lFpxk11WEa678](https://coderzcolumn.com/tutorials/data-science/simple-dashboard-with-widgets-python-bokeh?fbclid=IwAR1qkq4v3yphKh45LFoZJQQOJGxkgXOGxumn5QKpBeDwd9lFpxk11WEa678)
* Data
  * Iris flower dataset: comes with scikit-learn
  * Google OHLC dataset: comes with Bokeh

## 1. Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.resources import INLINE

output_notebook(resources=INLINE)

## 2. Load dataset

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris['Flower Type'] = iris.target

df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower Type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
import bokeh
bokeh.sampledata.download()

from bokeh.sampledata.stocks import GOOG as google

df_google = pd.DataFrame(google)
df_google['date'] = pd.to_datetime(df_google['date'])

df_google.head()

Creating /root/.bokeh directory
Creating /root/.bokeh/data directory
Using data directory: /root/.bokeh/data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3171836 bytes)
   3171836 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (4816256 bytes)
   4816256 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapminder_life_e

,date,open,high,low,close,volume,adj_close
0,2004-08-19,100.00,104.06,95.96,100.34,22351900,100.34
1,2004-08-20,101.01,109.08,100.50,108.31,11428600,108.31
2,2004-08-23,110.75,113.48,109.05,109.40,9137200,109.40
3,2004-08-24,111.24,111.60,103.57,104.87,7631300,104.87
4,2004-08-25,104.96,108.00,103.88,106.00,4598900,106.00


## 3. Creating individual charts of dashboard

### 3.1 Line chart of Google stock prices

In [4]:
line_chart = figure(
    plot_width=1000,
    plot_height=400,
    x_axis_type='datetime',
    title='Google stock prices from 2005 - 2013'
)

line_chart.line(
    x='date',
    y='open',
    line_width=0.5,
    line_color='dodgerblue',
    legend_label='open',
    source=df_google
)

line_chart.xaxis.axis_label='Time'
line_chart.yaxis.axis_label='Price ($)'
line_chart.legend.location='top_left'

show(line_chart)

Output hidden; open in https://colab.research.google.com to view.

### 3.2 Scatter plot of Iris flower sepal length vs sepal width color-encoded by flower type

In [5]:
scatter = figure(
    plot_width=500,
    plot_height=400,
    title='Sepal length vs sepal width scatter plot'
)

color_mapping = {
    0: 'tomato',
    1: 'dodgerblue',
    2: 'lime'
}

for cls in [0, 1, 2]:
    scatter.circle(
        x=df_iris[df_iris['Flower Type']==cls]['sepal length (cm)'],
        y=df_iris[df_iris['Flower Type']==cls]['sepal width (cm)'],
        color=color_mapping[cls],
        size=10,
        alpha=0.8,
        legend_label=iris.target_names[cls]
    )

scatter.xaxis.axis_label='sepal length (cm)'.upper()
scatter.yaxis.axis_label='sepal width (cm)'.upper()

show(scatter)

Output hidden; open in https://colab.research.google.com to view.

### 3.3 Bar chart of average sepal length per flower type

In [6]:
iris_avg_by_flower_type = df_iris.groupby('Flower Type').mean()

bar_chart = figure(
    plot_width=500,
    plot_height=400,
    title='Average sepal length (cm) per flower type'
)

bar_chart.vbar(
    x=[1, 2, 3],
    width=0.9,
    top=iris_avg_by_flower_type['sepal length (cm)'],
    fill_color='tomato',
    line_color='tomato',
    alpha=0.9
)

bar_chart.xaxis.axis_label='Flower type'
bar_chart.yaxis.axis_label='Sepal length'

bar_chart.xaxis.ticker = [1, 2, 3]
bar_chart.xaxis.major_label_overrides = {
    1: 'Setosa',
    2: 'Versicolor',
    3: 'Virginica'
}

show(bar_chart)

Output hidden; open in https://colab.research.google.com to view.

## 4 Laying out charts without widgets

In [7]:
layout = column(line_chart, row(scatter, bar_chart))
show(layout)

Output hidden; open in https://colab.research.google.com to view.

## 5. Widgets creation

### 5.1 Checkbox group for line chart

In [8]:
from bokeh.models import CheckboxButtonGroup

checkbox_options = ['open', 'high', 'low', 'close'] # 要顯示什麼

checkbox_grp = CheckboxButtonGroup(
    labels=checkbox_options,
    active=[0], # 預設選取了 checkbox_options 第 0 個 item 
    button_type='success'
)

show(checkbox_grp)

Output hidden; open in https://colab.research.google.com to view.

### 5.2 Dropdowns for scatter chart

In [9]:
from bokeh.models import Select

drop_scat1 = Select(
    title='X-axis-dim',
    options=iris.feature_names, # 下拉式選單中有什麼可以選
    value=iris.feature_names[0], # 預設選取的是哪個
    width=200
)

drop_scat2 = Select(
    title='Y-axis-dim',
    options=iris.feature_names, # 下拉式選單中有什麼可以選
    value=iris.feature_names[1], # 預設選取的是哪個
    width=200
)

show(row(drop_scat1, drop_scat2))

Output hidden; open in https://colab.research.google.com to view.

### 5.3 Dropdown for bar chart

In [10]:
drop_bar = Select(
    title='Dimension',
    options=iris.feature_names,
    value=iris.feature_names[0]
)

show(drop_bar)

Output hidden; open in https://colab.research.google.com to view.

## 6. Laying out charts & widget to create dashboard layout


In [11]:
layout_with_widgets = column(
    column(checkbox_grp, line_chart),
    row(
        column(row(drop_scat1, drop_scat2), scatter),
        column(drop_bar, bar_chart)
    )
)

show(layout_with_widgets)

Output hidden; open in https://colab.research.google.com to view.

# 7. Accessing individual elements of dashboard layout with indexing

In [12]:
show(layout_with_widgets.children[1])

Output hidden; open in https://colab.research.google.com to view.

In [13]:
show(layout_with_widgets.children[1].children[0])

Output hidden; open in https://colab.research.google.com to view.

## 8. Callbacks creation & widget attribute registration with callback

### 8.1 line chart callback and checkbox attribute registration with callback

In [14]:
def update_line_chart(attrname, old, new): # callback 函數的參數就是 (屬性, 舊的值, 新的值)
    '''
    Code to update line chart as per check box selection
    '''
    line_chart = figure(
        plot_width=1000,
        plot_height=400,
        x_axis_type='datetime',
        title='Google stock prices from 2005-2013'
    )

    price_color_map = {
        'open': 'dodgerblue',
        'close': 'tomato',
        'low': 'lime',
        'high': 'orange'
    }

    for option in checkbox_grp.active:
        line_chart.line(
            x='date',
            y=checkbox_options[option],
            line_width=0.5,
            line_color=price_color_map[checkbox_options[option]],
            legend_label=checkbox_options[option],
            source=df_google
        )

    line_chart.xaxis.axis_label='Time'
    line_chart.yaxis.axis_label='Price ($)'
    line_chart.legend.location='top_left'
    layout_with_widgets.children[0].children[1]=line_chart

# 要用 on_change() 來把 callback function 註冊
# 當 active 屬性改變的時候就呼叫 update_line_chart
checkbox_grp.on_change('active', update_line_chart)

### 8.2 Scatter chart callback and dropdowns attributes registration with callback

In [15]:
def update_scatter(attrname, old, new):
    '''
    Code to update scatter chart as per dropdown selections
    '''
    scatter = figure(
        plot_width=500,
        plot_height=400,
        title='%s vs %s Scatter Plot' % (drop_scat1.value.upper(), drop_scat2.value.upper())
    )

    for cls in [0, 1, 2]:
        scatter.circle(
            x=df_iris[df_iris['Flower Type']==cls][drop_scat1.value],
            y=df_iris[df_iris['Flower Type']==cls][drop_scat2.value],
            color=color_mapping[cls],
            size=10,
            alpha=0.8,
            legend_label=iris.target_names[cls]
        )

    scatter.xaxis.axis_label=drop_scat1.value.upper()
    scatter.yaxis.axis_label=drop_scat2.value.upper()

    layout_with_widgets.children[1].children[0].childern[1]=scatter

# 註冊 callback function
# 當 value 屬性改變的時候就呼叫 update_scatter
drop_scat1.on_change('value', update_scatter)
drop_scat2.on_change('value', update_scatter)

### 8.3 Bar chart callback and dropdown attribute registration with callback

In [16]:
def update_bar_chart(attrname, old, new):
    '''
    Code to update bar chart as per dropdown selections
    '''
    bar_chart = figure(
        plot_width=500,
        plot_height=400,
        title='Average %s per flower type' % drop_bar.value.upper()
    )

    bar_chart.vbar(
        x=[1, 2, 3],
        width=0.9,
        top=iris_avg_by_flower_type[drop_bar.value],
        fill_color='tomato',
        line_color='tomato',
        alpha=0.6
    )

    bar_chart.xaxis.axis_label='Flower Type'
    bar_chart.yaxis.axis_label=drop_bar.value.upper()

    bar_chart.xaxis.ticker=[1, 2, 3]
    bar_chart.xaxis.major_label_overrides = {
        1: 'Setosa',
        2: 'Versicolor',
        3: 'Virginica'
    }

    layout_with_widgets.children[1].children[1].children[1]=bar_chart

drop_bar.on_change('value', update_bar_chart)

## 9. Putting all together and bringing up dashboard

* 這是要寫在一個 python 檔案中，例如寫在 bokeh_dashboard.py

In [18]:
import pandas as pd

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import Select, CheckboxButtonGroup # Widgets

# Dataset imports
from bokeh.sampledata.stocks import GOOG as google
from sklearn.datasets import load_iris, load_wine

checkbox_options = ['open', 'high', 'low', 'close']
color_mapping = {
    0: 'tomato',
    1: 'dodgerblue',
    2: 'lime'
}
price_color_map = {
    'open': 'dodgerblue',
    'close': 'tomato',
    'low': 'lime',
    'high': 'orange'
}

# iris dataset loading
iris = load_iris()

df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris['Flower Type'] = iris.target

# Google price dataset loading
df_google = pd.DataFrame(google)
df_google['date'] = pd.to_datetime(df_google['date'])

# line chart of google price code starts
line_chart = figure(
    plot_width=1000,
    plot_height=400,
    x_axis_type='datetime',
    title='Google stock prices from 2005 - 2013'
)

line_chart.line(
    x='date',
    y='open',
    line_width=0.5,
    line_color='dodgerblue',
    legend_label='open',
    source=df_google
)

line_chart.xaxis.axis_label='Time'
line_chart.yaxis.axis_label='Price ($)'
line_chart.legend.location='top_left'

# Scatter chart of iris dimensions code starts
scatter = figure(
    plot_width=500,
    plot_height=400,
    title='Sepal length vs sepal width scatter plot'
)

for cls in [0, 1, 2]:
    scatter.circle(
        x=df_iris[df_iris['Flower Type']==cls]['sepal length (cm)'],
        y=df_iris[df_iris['Flower Type']==cls]['sepal width (cm)'],
        color=color_mapping[cls],
        size=10,
        alpha=0.8,
        legend_label=iris.target_names[cls]
    )

scatter.xaxis.axis_label='sepal length (cm)'.upper()
scatter.yaxis.axis_label='sepal width (cm)'.upper()

# Bar chart of iris dimensions code starts
iris_avg_by_flower_type = df_iris.groupby('Flower Type').mean()

bar_chart = figure(
    plot_width=500,
    plot_height=400,
    title='Average sepal length (cm) per flower type'
)

bar_chart.vbar(
    x=[1, 2, 3],
    width=0.9,
    top=iris_avg_by_flower_type['sepal length (cm)'],
    fill_color='tomato',
    line_color='tomato',
    alpha=0.9
)

bar_chart.xaxis.axis_label='Flower type'
bar_chart.yaxis.axis_label='Sepal length'

bar_chart.xaxis.ticker = [1, 2, 3]
bar_chart.xaxis.major_label_overrides = {
    1: 'Setosa',
    2: 'Versicolor',
    3: 'Virginica'
}

# Widgets code starts
drop_scat1 = Select(
    title='X-axis-dim',
    options=iris.feature_names, # 下拉式選單中有什麼可以選
    value=iris.feature_names[0], # 預設選取的是哪個
    width=225
)

drop_scat2 = Select(
    title='Y-axis-dim',
    options=iris.feature_names, # 下拉式選單中有什麼可以選
    value=iris.feature_names[1], # 預設選取的是哪個
    width=225
)

checkbox_grp = CheckboxButtonGroup(
    labels=checkbox_options,
    active=[0], # 預設選取了 checkbox_options 第 0 個 item 
    button_type='success'
)

drop_bar = Select(
    title='Dimension',
    options=iris.feature_names,
    value=iris.feature_names[0]
)

# Code to update charts as per widget state starts
def update_line_chart(attrname, old, new): # callback 函數的參數就是 (屬性, 舊的值, 新的值)
    '''
    Code to update line chart as per check box selection
    '''
    line_chart = figure(
        plot_width=1000,
        plot_height=400,
        x_axis_type='datetime',
        title='Google stock prices from 2005-2013'
    )

    for option in checkbox_grp.active:
        line_chart.line(
            x='date',
            y=checkbox_options[option],
            line_width=0.5,
            line_color=price_color_map[checkbox_options[option]],
            legend_label=checkbox_options[option],
            source=df_google
        )

    line_chart.xaxis.axis_label='Time'
    line_chart.yaxis.axis_label='Price ($)'
    line_chart.legend.location='top_left'
    layout_with_widgets.children[0].children[1]=line_chart

def update_scatter(attrname, old, new):
    '''
    Code to update scatter chart as per dropdown selections
    '''
    scatter = figure(
        plot_width=500,
        plot_height=400,
        title='%s vs %s Scatter Plot' % (drop_scat1.value.upper(), drop_scat2.value.upper())
    )

    for cls in [0, 1, 2]:
        scatter.circle(
            x=df_iris[df_iris['Flower Type']==cls][drop_scat1.value],
            y=df_iris[df_iris['Flower Type']==cls][drop_scat2.value],
            color=color_mapping[cls],
            size=10,
            alpha=0.8,
            legend_label=iris.target_names[cls]
        )

    scatter.xaxis.axis_label=drop_scat1.value.upper()
    scatter.yaxis.axis_label=drop_scat2.value.upper()

    layout_with_widgets.children[1].children[0].childern[1]=scatter

def update_bar_chart(attrname, old, new):
    '''
    Code to update bar chart as per dropdown selections
    '''
    bar_chart = figure(
        plot_width=500,
        plot_height=400,
        title='Average %s per flower type' % drop_bar.value.upper()
    )

    bar_chart.vbar(
        x=[1, 2, 3],
        width=0.9,
        top=iris_avg_by_flower_type[drop_bar.value],
        fill_color='tomato',
        line_color='tomato',
        alpha=0.9
    )

    bar_chart.xaxis.axis_label='Flower Type'
    bar_chart.yaxis.axis_label=drop_bar.value.upper()

    bar_chart.xaxis.ticker=[1, 2, 3]
    bar_chart.xaxis.major_label_overrides = {
        1: 'Setosa',
        2: 'Versicolor',
        3: 'Virginica'
    }

    layout_with_widgets.children[1].children[1].children[1]=bar_chart

# Regustering widget attribute change with methods code starts
checkbox_grp.on_change('active', update_line_chart)

drop_scat1.on_change('value', update_scatter)
drop_scat2.on_change('value', update_scatter)

drop_bar.on_change('value', update_bar_chart)

# Widget layout
layout_with_widgets = column(
    column(checkbox_grp, line_chart),
    row(
        column(row(drop_scat1, drop_scat2), scatter),
        column(drop_bar, bar_chart)
    )
)

# Creating dashboard
curdoc().add_root(layout_with_widgets)

## 10. Bringing up dashboard

In [19]:
!bokeh serve --show bokeh_dashboard.py

ERROR: Path for Bokeh server application does not exist: /content/bokeh_dashboard.py
